In [10]:
import struct
import pandas as pd
import numpy as np
import re

In [11]:
# file = pd.read_csv("pingtestdecode.txt", sep='-')
file = open("pingtestdecode.txt",'r')
# firstCol = np.asarray(file[0])
# print(firstCol)

a = file.read()

a.find('7e')

# get indices
indices = [m.start() for m in re.finditer('7e', a)]

packet_list = []
for index in range(len(indices) - 1):
    packet = a[indices[index]:indices[index+1] + 2]
    b = packet.split(sep='-')
    for i in range(len(b)):
        b[i]  = int('0x' + b[i],16)
    packet_list.append(b)
    
packet_list[0]

[126, 251, 48, 24, 123, 252, 126]

In [12]:

# CRC-CCITT calculation function for checking packet integrity
CRC_POLY = 0x1021

def crc_ccitt(data):
    crc = 0
    for byte in data:
        crc ^= byte << 8
        for _ in range(8):
            if crc & 0x8000:
                crc = (crc << 1) ^ CRC_POLY
            else:
                crc <<= 1
        crc &= 0xFFFF
    return crc

# Unescape and deframe the packet
def deframe_packet(packet):
    FLAG = 0x7E
    ESCAPE = 0x7D

    if packet[0] != FLAG or packet[-1] != FLAG:
        raise ValueError("Invalid packet framing.")
    
    # Remove flag bytes and unescape
    packet = packet[1:-1]
    unescaped = bytearray()
    i = 0
    while i < len(packet):
        if packet[i] == ESCAPE:
            unescaped.append(packet[i + 1] ^ 0x20)
            i += 2
        else:
            unescaped.append(packet[i])
            i += 1
    return unescaped

# Parse messages based on ID
def parse_message(data):
    msg_id = data[0]
    if msg_id == 0x00:  # Heartbeat
        gnss_valid = (data[1] & 0x80) != 0
        maintenance_required = (data[1] & 0x40) != 0
        timestamp = struct.unpack(">H", data[3:5])[0]
        print(f"Heartbeat - GNSS Valid: {gnss_valid}, Maintenance Required: {maintenance_required}, Timestamp: {timestamp}")

    elif msg_id == 0x0A:  # Ownship Report
        lat = int.from_bytes(data[5:8], "big", signed=True) * (180 / (2**23))
        lon = int.from_bytes(data[8:11], "big", signed=True) * (180 / (2**23))
        altitude = ((int.from_bytes(data[11:13], "big") >> 4) * 25) - 1000
        print(f"Ownship Report - Latitude: {lat}, Longitude: {lon}, Altitude: {altitude}")

    elif msg_id == 0x2F:  # Transponder Status
        mode_a_enabled = (data[2] & 0x10) != 0
        latitude = int.from_bytes(data[3:6], "big", signed=True) * (180 / (2**23))
        longitude = int.from_bytes(data[6:9], "big", signed=True) * (180 / (2**23))
        print(f"Transponder Status - Mode A Enabled: {mode_a_enabled}, Latitude: {latitude}, Longitude: {longitude}")

    else:
        print("Unknown Message ID")

# Main function to process the received packet
def process_packet(packet):
    try:
        data = deframe_packet(packet)
        # Check CRC (last 2 bytes)
        if crc_ccitt(data[:-2]) != int.from_bytes(data[-2:], "big"):
            raise ValueError("CRC mismatch")
        parse_message(data)
    except ValueError as e:
        print(f"Packet error: {e}")

In [15]:
# received_packet = b'\x7E\x00\x81\x00\x7D\x5E\x45\x12\x7E'  # Replace with actual received data
received_packet = bytearray(packet_list[0])
process_packet(received_packet)

for p in packet_list:
    received_packet = bytearray(p)
    process_packet(received_packet)

Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
Packet error: CRC mismatch
P